# 最終課題

In [6]:
# Webスクレイピングに最低限必要なライブラリをインポート
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

In [7]:
# --- 1. 設定セクション ---
# クローリングの基点URL
BASE_URL = 'https://www.musashino-u.ac.jp/'
# サーバーへの負荷軽減のための待機時間（秒）
SLEEP_TIME = 1 
# 収集対象外とするファイル拡張子、pdfとpngなど
EXCLUDED_EXTENSIONS = (
    '.pdf', '.png', '.jpg', '.jpeg', '.gif', '.zip', 
    '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', 
    '.mp4', '.mp3', '.css', '.js', '.ico','.webp'
)

# --- 2. データ構造 ---
unvisited_urls = [BASE_URL]
visited_urls = set()
# dict()関数を使って辞書を初期化
sitemap_data = dict() # 課題の提出形式: {URL: TITLE}
# page_count = 0 

print(f"--- クローリング開始 ---")
print(f"対象URL: {BASE_URL}")


# --- 3. メインクローリングロジック ---
while unvisited_urls:
    current_url = unvisited_urls.pop(0)

    if current_url in visited_urls: # 重複チェック
        continue
    
    visited_urls.add(current_url) # 訪問済みに追加
    time.sleep(SLEEP_TIME)

    print(f"url: {current_url}")
    
    # 【注意】ここで接続エラーが発生するとプログラムが停止します
    response = requests.get(current_url, timeout=10, 
                            headers={'User-Agent': 'MusashinoUniversityCrawler/1.0'})
    
    # ステータスコードのチェックと404スキップの明示
    if response.status_code != 200:
        if response.status_code == 404:
            print(f"スキップ: {current_url} - 404 Not Found (ページなし)")
        else:
            print(f"失敗: {current_url} - ステータスコード {response.status_code}")
        continue 

    # ページ取得が成功した場合の処理
    response.encoding = 'utf-8' 
    soup = BeautifulSoup(response.text, 'html.parser')

    # 1. <title>を取得し、辞書に格納
    title_tag = soup.find('title')
    page_title = title_tag.text.strip() if title_tag else "タイトルなし"
    sitemap_data[current_url] = page_title
    
    # page_count += 1 
    print(f"title: {page_title}")

    # 2. ページ内の全てのリンクを抽出
    for link in soup.find_all('a', href=True):
        href = link['href']
        absolute_url = urljoin(current_url, href)
        
        # URLを正規化
        parsed_url = urlparse(absolute_url)
        clean_url = parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path
        
        # ファイル拡張子による除外フィルタリング
        if clean_url.lower().endswith(EXCLUDED_EXTENSIONS):
            continue 

        # 3. 同一ドメイン内かつ未訪問のURLのみをリストに追加
        if parsed_url.netloc == urlparse(BASE_URL).netloc: 
            if clean_url not in visited_urls and clean_url not in unvisited_urls:
                unvisited_urls.append(clean_url)

print("\n--- クローリング終了 ---")

# --- 4. 結果の表示 ---
print("\n--- サイトマップ結果 ---")

final_count = 0
for url, title in sitemap_data.items():
    final_count += 1
    print(f"[{final_count:03}] TITLE: {title}")
    print(f"      URL: {url}")

print("\n--- 辞書型変数（生のデータ） ---")
# 課題の要件：辞書型変数を print() で表示
print(sitemap_data)

print(f"\n合計ページ数: {len(sitemap_data)} 件")

--- クローリング開始 ---
対象URL: https://www.musashino-u.ac.jp/
url: https://www.musashino-u.ac.jp/
title: 武蔵野大学
url: https://www.musashino-u.ac.jp/access.html
title: 交通アクセス | 武蔵野大学
url: https://www.musashino-u.ac.jp/admission/request.html
title: 資料請求 | 入試情報 | 武蔵野大学
url: https://www.musashino-u.ac.jp/contact.html
title: お問い合わせ | 武蔵野大学
url: https://www.musashino-u.ac.jp/prospective-students.html
title: 武蔵野大学で学びたい方 | 武蔵野大学
url: https://www.musashino-u.ac.jp/students.html
title: 在学生の方 | 武蔵野大学
url: https://www.musashino-u.ac.jp/alumni.html
title: 卒業生の方 | 武蔵野大学
url: https://www.musashino-u.ac.jp/parents.html
title: 保護者の方 | 武蔵野大学
url: https://www.musashino-u.ac.jp/business.html
title: 企業・研究者の方 | 武蔵野大学
url: https://www.musashino-u.ac.jp/guide/
title: 大学案内 | 武蔵野大学
url: https://www.musashino-u.ac.jp/guide/profile/
title: 大学紹介 | 大学案内 | 武蔵野大学
url: https://www.musashino-u.ac.jp/guide/activities/
title: 大学の取り組み | 大学案内 | 武蔵野大学
url: https://www.musashino-u.ac.jp/guide/campus/
title: キャンパス | 大学案内 | 武蔵野大学
url: 